In [1]:
#Compute limits
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Jan  2 16:43:30 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install transformers
!pip install accelerate
!pip install datasets
!pip install rouge_score
!pip install peft
!pip install trl
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 15.1 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import statistics

In [18]:
import os
import transformers
import torch
from datasets import load_dataset, Dataset, DatasetDict
from trl import SFTTrainer
from peft import LoraConfig, PeftModel, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer

# Download Gemma 2b-it base model
model_id = "google/gemma-2b-it"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
#model = AutoModelForCausalLM.from_pretrained(model_id,
#                                            quantization_config = bnb_config,
#                                            device_map={"":0})

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [6]:
# Configure LoRA
lora_config = LoraConfig(
    r = 8,
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
                      "gate_proj", "up_proj", "down_proj"],
    task_type = "CAUSAL_LM"
)

In [12]:
#dftr = pd.read_csv('/content/stories_csv_bs.csv')
dftr = pd.read_excel('/content/stories_excel_bs_v1.xlsx')
dftr

,Prompt,Title,Part,Story
0,గాడిద జోస్యం అనే చందమామ కథ చెప్పు?,గాడిద జ్యోస్యం,ఒకటి,సువర్ణానదీ తీరాన ఒక గ్రామంలో ఒక జ్యోతిష్కుడుండ...
1,NaN,గాడిద జ్యోస్యం,రెండు,ఆ సమయానికి ఊరి చాకలి ఉతికిన బట్టలు గాడిద పైన వ...
2,NaN,గాడిద జ్యోస్యం,మూడు,"సిద్ధాంతి గారికి ఏమి చేయాలో తోచలేదు, చాకలి చెప..."
3,ఓవర్ కాన్ఫిడెన్స్ ఉండకూడదు అనే దానికి ఉదాహరణగ...,గాడిద జ్యోస్యం,ఒకటి,సువర్ణానదీ తీరాన ఒక గ్రామంలో ఒక జ్యోతిష్కుడుండ...
4,బ్రహ్మప్రళయం గురించి ఒక చందమామ కథ చెప్పు?,బ్రహ్మ ప్రళయం,ఒకటి,"బ్రహ్మదేవుడు భూమిపై మానవులను సృష్టించి, నాయనలా..."
...,...,...,...,...
456,NaN,నిజమైన చదువు,రెండు,ఈ రాజకుమారుడు తలుచుకుంటే తనను కొరతవేయించగల డను...
457,చదువుకోవడం గురించి కథ చెప్పు?,నిజమైన చదువు,ఒకటి,ఒక గురువు దగ్గిర అనేకమంది విద్యార్థులు చదువుకు...
458,రాకుమారుడి కథ చెప్పు?,నిజమైన చదువు,ఒకటి,ఒక గురువు దగ్గిర అనేకమంది విద్యార్థులు చదువుకు...
459,అతితెలివి గురించి కథ చెప్పు?,అతితెలివి,ఒకటి,ఒక ఆసామి తన పొరుగు ఆసామివద్ద ఒక కుండ అరువుతిసు...


In [13]:
dftr = dftr.dropna(subset=["Title"])
dftr

,Prompt,Title,Part,Story
0,గాడిద జోస్యం అనే చందమామ కథ చెప్పు?,గాడిద జ్యోస్యం,ఒకటి,సువర్ణానదీ తీరాన ఒక గ్రామంలో ఒక జ్యోతిష్కుడుండ...
1,NaN,గాడిద జ్యోస్యం,రెండు,ఆ సమయానికి ఊరి చాకలి ఉతికిన బట్టలు గాడిద పైన వ...
2,NaN,గాడిద జ్యోస్యం,మూడు,"సిద్ధాంతి గారికి ఏమి చేయాలో తోచలేదు, చాకలి చెప..."
3,ఓవర్ కాన్ఫిడెన్స్ ఉండకూడదు అనే దానికి ఉదాహరణగ...,గాడిద జ్యోస్యం,ఒకటి,సువర్ణానదీ తీరాన ఒక గ్రామంలో ఒక జ్యోతిష్కుడుండ...
4,బ్రహ్మప్రళయం గురించి ఒక చందమామ కథ చెప్పు?,బ్రహ్మ ప్రళయం,ఒకటి,"బ్రహ్మదేవుడు భూమిపై మానవులను సృష్టించి, నాయనలా..."
...,...,...,...,...
456,NaN,నిజమైన చదువు,రెండు,ఈ రాజకుమారుడు తలుచుకుంటే తనను కొరతవేయించగల డను...
457,చదువుకోవడం గురించి కథ చెప్పు?,నిజమైన చదువు,ఒకటి,ఒక గురువు దగ్గిర అనేకమంది విద్యార్థులు చదువుకు...
458,రాకుమారుడి కథ చెప్పు?,నిజమైన చదువు,ఒకటి,ఒక గురువు దగ్గిర అనేకమంది విద్యార్థులు చదువుకు...
459,అతితెలివి గురించి కథ చెప్పు?,అతితెలివి,ఒకటి,ఒక ఆసామి తన పొరుగు ఆసామివద్ద ఒక కుండ అరువుతిసు...


In [14]:
dftr['Extract_Chunk'] = dftr['Story'].apply(lambda x: '' if x.split('.')[-2] == '\nకథ సమాప్తం'
                                            else ''.join([l.strip() + '. ' for l in x.split('.')[-2:-1]]))
dftr['Extract_Chunk']

,Extract_Chunk
0,"మధ్యాహ్న భోజనం అయాక ఒక్ష కునుకు తీసి లేచి, సిద..."
1,"ఆకాశాన ఉరుములూ, మెరుపులూ సాగి వాన ఆరంభమయింది."
2,
3,"మధ్యాహ్న భోజనం అయాక ఒక్ష కునుకు తీసి లేచి, సిద..."
4,మరుక్షణం బ్రహ్మ అంతర్థానమయిపోయాడు.
...,...
456,"నిజమైన చదువంటే పాఠాలు కంఠస్థం చెయ్యటం కాదు, వా..."
457,అంతలోనే గురువుకు తాను కొడుతున్నది మామూలు కుర్ర...
458,అంతలోనే గురువుకు తాను కొడుతున్నది మామూలు కుర్ర...
459,ఆసామి ప్లీడరువద్దకు వెళ్లి నలహా అడిగాడు.


In [15]:
dftr['Extract_Chunk'][1]

'ఆకాశాన ఉరుములూ, మెరుపులూ సాగి వాన ఆరంభమయింది. '

In [16]:
for i in range(len(dftr)):
  if dftr.at[i, 'Part'] == 'ఒకటి':
    dftr.at[i,'Context'] = ''
    dftr.at[i,'To_Tokenize'] = 'Prompt: ' + dftr.at[i,'Prompt'] + '\n' + 'Title: ' + dftr.at[i, 'Title'] + '\n' + 'Story: ' + dftr.at[i, 'Story'] + '<eos>'

  elif dftr.at[i, 'Extract_Chunk'] == '':
    dftr.at[i,'Context'] = dftr.at[i-1,'Extract_Chunk']
    dftr.at[i, 'To_Tokenize'] = 'Prompt: ' + dftr.at[i, 'Context']+ '\n' + dftr.at[i, 'Story'] + '<eos>'

  else:
    dftr.at[i,'Context'] = dftr.at[i-1,'Extract_Chunk']
    dftr.at[i, 'To_Tokenize'] = 'Prompt: ' + dftr.at[i, 'Context']+ '\n' + dftr.at[i, 'Story'] + '<eos>'

dftr['To_Tokenize']

,To_Tokenize
0,Prompt: గాడిద జోస్యం అనే చందమామ కథ చెప్పు?\nTi...
1,Prompt: మధ్యాహ్న భోజనం అయాక ఒక్ష కునుకు తీసి ల...
2,"Prompt: ఆకాశాన ఉరుములూ, మెరుపులూ సాగి వాన ఆరంభ..."
3,Prompt: ఓవర్ కాన్ఫిడెన్స్ ఉండకూడదు అనే దానికి...
4,Prompt: బ్రహ్మప్రళయం గురించి ఒక చందమామ కథ చెప...
...,...
456,Prompt: అంతలోనే గురువుకు తాను కొడుతున్నది మామూ...
457,Prompt: చదువుకోవడం గురించి కథ చెప్పు?\nTitle:...
458,Prompt: రాకుమారుడి కథ చెప్పు?\nTitle: నిజమైన చ...
459,Prompt: అతితెలివి గురించి కథ చెప్పు?\nTitle: అ...


In [19]:
dftr['Story_len'] = dftr['To_Tokenize'].apply(lambda x: len(tokenizer(x)['input_ids']))
dftr['Story_len'].describe()

,Story_len
count,461.000000
mean,628.308026
std,142.505151
min,284.000000
25%,526.000000
50%,619.000000
75%,728.000000
max,992.000000


In [67]:
dftr.to_csv("output.csv", index=False)

In [20]:
dftr[dftr['Story_len'] > 900]

,Prompt,Title,Part,Story,Extract_Chunk,Context,To_Tokenize,Story_len
9,NaN,అప్పు తెచ్చిన ముప్పు,మూడు,\nఒకరోజు ప్రభుత్వాధికారులు కొందరు హఠాత్తుగా రా...,,భర్తపైన అలిగి పుట్టింటికి వెళ్లిపోయింది సీతాలక...,Prompt: భర్తపైన అలిగి పుట్టింటికి వెళ్లిపోయింద...,949
26,NaN,వ్యత్యాసం,రెండు,మర్నాడుఉదయం భార్యా భర్తలిద్దరూ కల్యాణవర్మ ఇంటి...,,"దానికేంభాగ్యం రేపు నా వెంటరా, ఇద్దరి ఇళ్లకూ వె...","Prompt: దానికేంభాగ్యం రేపు నా వెంటరా, ఇద్దరి ఇ...",923
48,NaN,అర్దణా రాజు,రెండు,మిట్టమధ్యాహ్నం కావస్తోంది. నెత్తిన ఎండ మందడివో...,"అందుచేత త్వరగా నన్ను పోనివ్వండి, అన్నాడు నీళ్ల...",నేనిప్పుడే ఉత్తర ద్వారానికి వెళ్ళి నా అర్ధణాతె...,Prompt: నేనిప్పుడే ఉత్తర ద్వారానికి వెళ్ళి నా ...,972
78,NaN,బామ్మగారు,మూడు,సాయంత్రం వరకు పిల్లలతో గడిపి అందరితో పాటు లాడ్...,ఎవరిళ్లకు వారు చేరుకున్నారు.,చేతులు శుభ్రం చేసుకున్నప్పుడు చొక్కా పూర్తిగా ...,Prompt: చేతులు శుభ్రం చేసుకున్నప్పుడు చొక్కా ప...,931
91,చెట్టు గొప్పతనం గురించి కథ చెప్పు?,సజీవ దేవుడు,ఒకటి,భర్త రాము పనీపాటా లేకుండా తోటలో కూర్చుని ఉండటం...,అలా ఆలోచిస్తూ భుజాలు ఎగరేశాడు రాము.,,Prompt: చెట్టు గొప్పతనం గురించి కథ చెప్పు?\nTi...,929
92,NaN,సజీవ దేవుడు,రెండు,అనంతరం పట్టణం వెళ్లి కలప వ్యాపారిని కలిశాడు. త...,వరద ప్రమాదంనుంచి గట్టెక్కినట్లే.,అలా ఆలోచిస్తూ భుజాలు ఎగరేశాడు రాము.,Prompt: అలా ఆలోచిస్తూ భుజాలు ఎగరేశాడు రాము. \n...,935
93,NaN,సజీవ దేవుడు,మూడు,కాస్సేపయ్యాక తను ఒక చెట్టును పట్టుకుని ఉన్నట్ల...,ఈ ఆలోచన రాగానే రాము మనసు తేలికైంది.,వరద ప్రమాదంనుంచి గట్టెక్కినట్లే.,Prompt: వరద ప్రమాదంనుంచి గట్టెక్కినట్లే. \nకాస...,936
95,వృక్షో రక్షతి రక్షితః అన్నదానికి కథ చెప్పు?,సజీవ దేవుడు,ఒకటి,భర్త రాము పనీపాటా లేకుండా తోటలో కూర్చుని ఉండటం...,ఈ పనిలో సాయపడినవారికి తలాకొంత పంచి మిగిలిన సొమ...,,Prompt: వృక్షో రక్షతి రక్షితః అన్నదానికి కథ చ...,911
98,NaN,గురువుగారి ఆంతర్యం,మూడు,విజయుడు పశ్చిమ దిశ వైపు వేగంగా కదిలాడు. ఇక్కడ ...,విగ్రహాన్ని అక్కడే జారవిడచి ముందుకు దూకి పాపను...,"తమరు ముందుకు సాగిపొండి, అన్నాడు సుబుద్ధి.","Prompt: తమరు ముందుకు సాగిపొండి, అన్నాడు సుబుద్...",990
99,NaN,గురువుగారి ఆంతర్యం,నాలుగు,అయితే విగ్రహం చెయ్యి విరిగి అతని కష్టమంతా కళ్ల...,,విగ్రహాన్ని అక్కడే జారవిడచి ముందుకు దూకి పాపను...,Prompt: విగ్రహాన్ని అక్కడే జారవిడచి ముందుకు దూ...,903


In [37]:
print(dftr['Part'].unique())
len(dftr['Part'].unique())

['ఒకటి' 'రెండు' 'మూడు' 'నాలుగు' 'ఐదు' 'ఆరు' 'ఏడు' 'ఎనిమిది' 'తొమ్మిది']


9

In [69]:
import json
train_data = []
for index, row in dftr[:22].iterrows():
  train_data += [{'prompt': row.iloc[6]}]
with open('train_file.json', 'w') as file:
    json.dump(train_data, file)

In [70]:
# Tokenize the Dataset

from datasets import load_dataset
dataset = load_dataset('json', data_files = {'train':'train_file.json'})
tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples):

    inputs = [pro for pro in examples['prompt']]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length")
    labels = model_inputs['input_ids'].copy()
    model_inputs['labels'] = labels
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function,  batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/22 [00:00<?, ? examples/s]

In [ ]:
import gc
torch.cuda.empty_cache()
del dftr
gc.collect()

155

In [71]:
from transformers import Trainer, TrainingArguments

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Training arguments configuration
training_args = TrainingArguments(
    output_dir="./results",
    #evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    learning_rate=2e-4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=15,
    weight_decay=0.01,
    save_total_limit=1,
    logging_dir='./logs',
    logging_strategy='epoch',
    save_strategy = 'epoch',
    report_to="none",
    fp16=True,
    optim="adamw_torch"
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    #eval_dataset=tokenized_dataset,  # Ideally, separate eval dataset
)

trainer.train()

Step,Training Loss
22,2.042400
44,1.424700
66,0.978000
88,0.581300
110,0.335000
132,0.171500
154,0.081500
176,0.045300
198,0.025700
220,0.016500


TrainOutput(global_step=330, training_loss=0.38290037792740444, metrics={'train_runtime': 374.3249, 'train_samples_per_second': 0.882, 'train_steps_per_second': 0.882, 'total_flos': 4038191795404800.0, 'train_loss': 0.38290037792740444, 'epoch': 15.0})

In [79]:
def generate_chunk(prompt, by_user=0):
  input_ids = tokenizer.encode("prompt:" + prompt + "\n", return_tensors='pt').to('cuda')
  attention_mask = torch.ones(input_ids.shape).to('cuda')
  attention_mask = torch.ones(input_ids.shape).to('cuda')
  output = model.generate(input_ids, attention_mask = attention_mask, max_new_tokens=1024)
  gen_text = tokenizer.decode(output[0], skip_special_tokens=True)
  if by_user == 1:
    prompt, gen_chunk = gen_text.split('Title:', maxsplit=1)
  else:
    prompt, gen_chunk = gen_text.split(prompt, maxsplit=1)
  return gen_chunk

In [80]:
def generate_full_story(user_prompt):
    story = 'Title:'
    story += generate_chunk(user_prompt, by_user=1)

    next_prompt = ''.join([l.strip() + '. ' for l in story.split('.')[-2:-1]])
    i = 1; max_generations = 9

    while story[-11:] != 'కథ సమాప్తం.':
      if i > max_generations:
        break
      gen_chunk = generate_chunk(next_prompt)
      i = i + 1
      story = story + '\n' + gen_chunk
      next_prompt = ''.join([l.strip() + '. ' for l in gen_chunk.split('.')[-2:-1]])

    return story


In [81]:
user_prompt = "బ్రహ్మప్రళయం గురించి ఒక చందమామ కథ చెప్పు?"
print(generate_full_story(user_prompt))

Title: బ్రహ్మ ప్రళయం
Story: బ్రహ్మదేవుడు భూమిపై మానవులను సృష్టించి, నాయనలారా, తిరిగి నేను ప్రళయం కలిగించేదాకా మీరు ఈ భూమిపై జీవించండి అని వరం ఇచ్చాడు.
దేవా తిరిగి ప్రళయం, ఎప్పుడు వస్తుందో మాకు తెలిసినట్టయితే మేము భూమిపై చేయదగిన కార్యాలను నిర్ణయించుకుంటాము అన్నారు మానవులు.
ప్రళయం ఎంతకాలానికి జరిగేదీ మీకు తెలిసేందుకు నేనొక ఏర్పాటు చేస్తాను, అంటూ బ్రహ్మదేవుడు ఒకచోట మూడు కర్రలు పాతాడు. మొదటి కరకు వలయాలు అమర్చాడు. అట్టడుగున ఉన్న వలయం అన్నిటి కన్నా పెద్దది, దానిపైది కొంచెం చిన్నది, దానిపైది ఇంకా కొంచెం చిన్నది, అన్ని వలయాలకూ పైన ఉన్నది అన్నిటికన్న చిన్నది.
తరవాత బ్రహ్మదేవుడు ముగ్గురినీ పిలిచి, మీరు బ్రహ్మప్రళయం దాకా జీవించే వరం ఇస్తున్నాను. మీ పని ఏమిటంటే ఈ వలయాలన్నిటినీ ఇదే క్రమంలో మూడవ కర్రకు మార్చాలి. పెద్ద వలయం మీద చిన్న వలయం ఉంచవచ్చునే గాని చిన్నదాని మీద పెద్ద వలయం ఉంచరాదు. రెండవ కర్రను తాత్కాలికంగా వలయాలుంచటానికి మాత్రవే ఉపయోగించాలి. మీరు ముగ్గురూ వంతులు వేసుకుని ఈ పని సాగించండి. ఈ అరవై నాలుగు వలయాలూ మూడవ కర్రకు ఇదే క్రమంలో అమర్చిన క్షణాన ప్రళయం వస్తుంది అన్నాడు. మరుక్షణం బ్రహ్మ అంతర్థ